# Prerpocesamiento de datos

#### En este post vamos a introducirnos en el preprocesamiento de datos utilizando como herramienta la librería Sklearn de Python 

#### ¿Qué es el preprocesamiento de datos?
##### A grandes rasgos es cualquier transformación que hagamos sobre los datos para que puedan ser utilizados. Por ejemplo, la eliminación o reemplazo de datos nulos, cambiar el tipo de dato de texto a datos numéricos, re-escalamientos, tratamiento de valores atípicos, etc. Es uno de os procesos más importantes porque de un buen trabajo de preprocesamiento dependerá mucho la calidad de los datos que alimentaran al modelo. En nuestro caso particular vamos a trabajar sobre algunas variables del dataset para que puedan ser consumidas por un algoritmo. Muchos de los algoritmos utilizados en machine learning necesitan que los datos, además de ser numéricos, sean re-escalados para que las diferencias en escala no afecten el resultado. Un ejemplo sería si queremos segmentar casas tomando las variables "precio" y "número de habitaciones". La escala de precios se ubicará en el rango de las decenas a las centenas de miles, mientras que el número de habitaciones difícilmente supere la decena. Si pensamos en estas variables como pares ordenados en un plano ya podríamos imaginar la distorsión que se crearía si los datos no están re-escalados. 

##### Para desarrollar el ejemplo vamos a utilizar la librería Sklearn de Python y en particular sl módulo Preprocessing

#### Primero cargamos las librerías y el dataset

In [21]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, StandardScaler, OneHotEncoder, LabelEncoder

df = pd.read_csv("Social_Network_Ads.csv")

df.head()

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0


##### Vamos a reescalar las variables _edad_ "Age" y _salario estimado_ "EstimatedSalary", primero utilizando MinMaxScaler y luego StandardScaler 

#### Normalización / MinMaxScaler

##### MinMaxScaler se utiliza para normalizar los datos, es decir que comprimirá los valores en un rango, en este caso entre 0 y 1 (o -1 y 1 si en la variable hay valores negativos), tomando el valor mínimo de la variable como 0 y el valor máximo como 1. La fórmula de normalización es la siguiente: /n 
$ Y = (X - min)/(max - min) $

##### Para aplicar MinMazScaler aplicamos los siguientes pasos:  
- crear el objeto escalador
- obtener el mínimo y máximo con la función fit()
- transformar nuestros datos fon la función transform()

##### Los dos últimos pasos se pueden unificar en la función fit_transfor(), que es la que vamos a utilizar. Se utiliza separados cuando tenemos datos de entrenamiento y datos de testeo, pero eso l vamos a ver en otro post

In [22]:
# Creo el objeto escalador
escalador_minmax = MinMaxScaler()

# Creo un nuevo dataframe solo porque debo aplicar otros métodos
df_minmax = df

# Aplico la función fit_transform() sobre las variables que quiero modificar y las reemplazo en el dataframe original
df_minmax[["Age", "EstimatedSalary"]] = escalador_minmax.fit_transform(df_minmax[["Age", "EstimatedSalary"]])

# Imprimo las primeras filas del dataframe original para ver cómo queda
df_minmax.head()

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,0.023810,0.029630,0
1,15810944,Male,0.404762,0.037037,0
2,15668575,Female,0.190476,0.207407,0
3,15603246,Female,0.214286,0.311111,0
4,15804002,Male,0.023810,0.451852,0


#### Estandarización / StandarScaler

##### La estandarización funciona re-escalando para transformar la distribución de los valores llevando la media a 0 y la desviación estándar a 1. La fórmula es la siguiente:
$ Y = (X - media)/desviación estandar $

#### La aplicación es muy igual a la del min max
- Crear el objeto escalador
- Se calculan los estadísticos con la función fit()
- Se aplica sobre los datos con la función transform()

##### Al igual que con min max en nuestro caso vamos a utilizar la función fit_transform()

In [23]:
# Creo el objeto escalador
escalador_std = StandardScaler()

# Creo un nuevo dataframe solo porque debo aplicar otros métodos
df_std = df

# Aplico la función fit_transform() sobre las variables que quiero modificar y las reemplazo en el dataframe original
df_std[["Age", "EstimatedSalary"]] = escalador_std.fit_transform(df_std[["Age", "EstimatedSalary"]])

# Imprimo las primeras filas del dataframe original para ver como queda
df_std.head()

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,-1.781797,-1.490046,0
1,15810944,Male,-0.253587,-1.460681,0
2,15668575,Female,-1.113206,-0.785290,0
3,15603246,Female,-1.017692,-0.374182,0
4,15804002,Male,-1.781797,0.183751,0


#### Transformar variables categóricas / LabelEncoder + OneHotEncoder

##### Cuando tenemos variables categóricas, en nuestro caso tenemos una columna de _genero_, y queremos utilizarla en nuestro modelo, necesitamos de alguna manera transformarlas en variables numéricas para que puedan ser consumidas por el algoritmo. Utilizando LabelEncoder y OneHotEncoder podemos crear tantas columnas como categorías tengamos (obviamente esto es útil solo cundo tenemos pocas categorías) para luego si poder utilizarlas

##### Lo primero que hacemos es utilizar el LabelEncoder para crear una nueva columna con variables numéricas

In [24]:
# Creo el objeto 
etiquetas = LabelEncoder()

# Un nuevo dataframe solo para practicidad en este caso
df_encoders = df

# Creamos los valores numéricos y los almacenamos en una nueva columna
df_encoders["etiquetas"] = etiquetas.fit_transform(df_encoders["Gender"])

# Vemos como queda
df_encoders.head()

,User ID,Gender,Age,EstimatedSalary,Purchased,etiquetas
0,15624510,Male,-1.781797,-1.490046,0,1
1,15810944,Male,-0.253587,-1.460681,0,1
2,15668575,Female,-1.113206,-0.785290,0,0
3,15603246,Female,-1.017692,-0.374182,0,0
4,15804002,Male,-1.781797,0.183751,0,1


##### Como los valores numéricos tienden a ser malinterpretados por los algoritmos aún necesitamos realizar un paso más. A partir de las nuevas etiquetas creamos nuevas columnas utilizando OneHotEncoder

In [25]:
# Creo el objeto
codificador = OneHotEncoder()

# Aplico la función fit_transform() pero necesitamos transformar el resultado en array y crear un nuevo dataframe 
# porque OneHotEncoder nos devuelve una matriz que no podemos utilizar directamente
ohe = pd.DataFrame(codificador.fit_transform(df_encoders[["etiquetas"]]).toarray())

# Unimos el nuevo dataframe generado al anterior para tener todas las columnas
df_ohe = df_encoders.join(ohe)

# Y vemos como queda
df_ohe.head()

,User ID,Gender,Age,EstimatedSalary,Purchased,etiquetas,0,1
0,15624510,Male,-1.781797,-1.490046,0,1,0.0,1.0
1,15810944,Male,-0.253587,-1.460681,0,1,0.0,1.0
2,15668575,Female,-1.113206,-0.785290,0,0,1.0,0.0
3,15603246,Female,-1.017692,-0.374182,0,0,1.0,0.0
4,15804002,Male,-1.781797,0.183751,0,1,0.0,1.0


##### Vemos que al final se sumaron dos nuevas columnas ahora sí listas para ser ingresadas en nuestro modelo

#### Este post obviamente no agota el tema del preprocesamiento pero es una buena introducción para tener un pantallazo de las funcionalidades que vamos a utilizar con más frecuencia. Hasta la próxima!